In [144]:
import pandas as pd
import streamlit
import itertools 
import logging
from math import ceil
import sys
sys.tracebacklimit = None


mlprep_sites = [1,2,3,4,5,6,7,8]
plate_size=24
key = pd.read_csv("data/manual_key_copy.txt",header=0, sep="\t",  quoting=3,keep_default_na=False)
key = key.applymap(lambda row: row.replace('"',""))
key = key.applymap(lambda row: row.replace(' ',""))
source=pd.read_csv("data/source_sheet2_with_site.csv",header=0,  quoting=3)
#print(key)
#print(source)
pgp_combos_pre = key.values.tolist()
pgp_combos_pre = [[element for element in sublist if element != ""] for sublist in pgp_combos_pre]
pgp_combos = [tuple(x) for x in pgp_combos_pre]
pgp_flat_combos = [proguide for sublist in pgp_combos for proguide in sublist]
pgp_flat_combos_split = [proguide for sublist in pgp_flat_combos for proguide in sublist.split(",")]
#keep track of the mlprep_sites
source_sites = list(set(source.site))
num_sourcesites= len(source_sites)
if (num_sourcesites > 2):
     raise ValueError("You can't use more than two sites in the MLprep for source plates.") 
mlprep_sites =  [x for x in mlprep_sites if x not in source_sites]
print(key)
print(source)
#print(source_sites)
#print(mlprep_sites)
#target_combos = [tuple(x) for x in source.values.tolist()]
#target_flat_combos = [target for sublist in target_combos for target in sublist]


            step1          step2          step3          step4
0          PGP001         PGP002                              
1          PGP001         PGP002                              
2          PGP001         PGP002                              
3          PGP001         PGP002                              
4          PGP001  PGP002,PGP005         PGP006         PGP007
5          PGP001  PGP002,PGP005  PGP003,PGP006               
6   PGP001,PGP008  PGP002,PGP005  PGP003,PGP006         PGP004
7   PGP001,PGP008  PGP002,PGP005  PGP003,PGP006  PGP004,PGP007
8   PGP001,PGP008  PGP002,PGP011  PGP003,PGP006  PGP004,PGP007
9          PGP001         PGP002         PGP003               
10         PGP001         PGP002         PGP003         PGP004
11         PGP001         PGP002         PGP003               
12         PGP001         PGP002         PGP003         PGP004
13         PGP001         PGP002         PGP003               
14         PGP001         PGP002         PGP003        

In [139]:
max_steps= len((key.columns))
num_pgps= len(set(pgp_flat_combos_split))
num_cellgos = len(key.index)

pgp_combos_fixed=[]
for element in pgp_combos_pre:
    the_list=[map(lambda x: x.strip(), item.split(',')) for item in element]
    my_tuple=tuple(item for sub_list in the_list for item in sub_list)
    pgp_combos_fixed.append(my_tuple)
pgp_combos_fixed_lengths= [len(item) for item in pgp_combos_fixed]

wells = [f'{letter}{i}'for letter in ['A','B','C','D','E','F']  for i in range(1, 5)]
wells_cycle=list(itertools.islice(itertools.cycle(wells), num_cellgos))
well_rows = list(itertools.chain.from_iterable([itertools.repeat(item, count) for item, count in zip(wells_cycle,pgp_combos_fixed_lengths)]))
num_targetsites = ceil(len(wells_cycle)/plate_size)
num_pipetsites = ceil(len(well_rows)/plate_size)

print(mlprep_sites)
if (( num_sourcesites + num_targetsites  + num_pipetsites) > 8):
    raise SystemExit(f"There are 8 total physical sites on the MLprep you can place plates. You are attempting to use {num_sourcesites + num_targetsites  + num_pipetsites} sites. Please lower the number of cellgorithms you are attempting to create to get below 8 total sites . # SourceSites you are attempting to use: {num_sourcesites}. # TargetSites you are attempting to use: {num_targetsites}. # PipetSites you are attempting to use: {num_pipetsites}")
target_sites = mlprep_sites[:num_targetsites]
mlprep_sites =  [x for x in mlprep_sites if x not in target_sites]
targetsites_sheet=[item for item in target_sites for i in range(plate_size)]
targetsites_sheet= targetsites_sheet[:len(wells_cycle)]
targetsites_rows = list(itertools.chain.from_iterable([itertools.repeat(item, count) for item, count in zip(targetsites_sheet,pgp_combos_fixed_lengths)]))
print(source_sites)
print(num_sourcesites)
print(num_targetsites)
print(num_pipetsites)
print(len(well_rows))
print(target_sites)
print(mlprep_sites)
print(wells_cycle)
print(len(wells_cycle))
print(len(targetsites_sheet))
print(targetsites_sheet)
print(targetsites_rows)
print(len(targetsites_rows))

[3, 4, 5, 6, 7, 8]
[1, 2]
2
2
4
84
[3, 4]
[5, 6, 7, 8]
['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'C1', 'C2', 'C3', 'C4', 'D1', 'D2', 'D3', 'D4', 'E1', 'E2', 'E3', 'E4', 'F1', 'F2', 'F3', 'F4', 'A1']
25
25
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4]
84


In [149]:
df = pd.DataFrame(columns=["SourceSite","SourceWell","TargetSite","TargetWell",'pgp'])
df['TargetWell'] = well_rows
df['TargetSite'] = targetsites_rows
df['pgp'] = pgp_flat_combos_split
idx = source.reset_index().set_index('pgp').loc[df.pgp, 'index'].values.tolist()
df['SourceSite']=source.site[idx].tolist()
df['SourceWell']=source.well[idx].tolist()

df = pd.merge(left=df,right=source[["pgp","target"]],how="left",on='pgp')


print(df.to_string())

    SourceSite SourceWell  TargetSite TargetWell     pgp target
0            1         A1           3         A1  PGP001  MEIS1
1            1         A2           3         A1  PGP002  MEIS1
2            1         A1           3         A2  PGP001  MEIS1
3            1         A2           3         A2  PGP002  MEIS1
4            1         A1           3         A3  PGP001  MEIS1
5            1         A2           3         A3  PGP002  MEIS1
6            1         A1           3         A4  PGP001  MEIS1
7            1         A2           3         A4  PGP002  MEIS1
8            1         A1           3         B1  PGP001  MEIS1
9            1         A2           3         B1  PGP002  MEIS1
10           1         A5           3         B1  PGP005  blank
11           1         A6           3         B1  PGP006   CD19
12           1         A7           3         B1  PGP007   PAX3
13           1         A1           3         B2  PGP001  MEIS1
14           1         A2           3   

In [22]:





df['SourceSite'] = 1
idx.sourcesite = source.reset_index().set_index('pgp').loc[df.pgp, 'index'].values.tolist()


df.to_csv('manual_output.txt',sep=',',index=False,quoting=None)
pgp_table = df.pgp.value_counts()
pgp_max=pgp_table.max()
pgp_min=pgp_table.min()
max_pgp_list=pgp_table[pgp_table == pgp_max].index.tolist()
min_pgp_list=pgp_table[pgp_table == pgp_min].index.tolist()
num_targets= len(set(df.target))
pipets_needed =len(df.index)
print(pgp_combos_fixed_lengths)
with open('manual.log', 'w'):
   pass
logging.basicConfig(filename='manual.log',
                            filemode='w',
                            format='%(asctime)s,%(message)s',
                            datefmt='%H:%M:%S',
                            level=logging.DEBUG,force=True)
logging.info(f'Max number of unique cellgorithm steps is {max_steps}')
logging.info(f'Number of proguides used is {num_pgps}')
logging.info(f'Number of unique targets is {num_targets}')
logging.info(f'Number of unique cellgos|unique TargetWells is {num_cellgos}')
logging.info(f'Total number of pipets needed/robotic transfers is {pipets_needed}')
logging.info(f'{max_pgp_list} proguide(s) use the most material. Each proguide(s) requires {pgp_max} transfers and {4  * pgp_max} ul minimum in the source plate (@ 4ul per transfer)')
logging.info(f'{min_pgp_list} proguide(s) use the least material require {pgp_min} transfers and {4 * pgp_min} ul minimum in the source plate')

125

84

In [28]:
key = pd.read_csv("data/key_no_target.csv", header=0, quoting=3)
source = pd.read_csv("data/source_sheet1_with_target.csv", header=0, quoting=3)
#key = pd.merge(left=key, right=source[["pgp", "target"]], how="left", on="pgp")
print(key)
print(source)

      pgp  step
0  PGP001     1
1  PGP002     2
2  PGP003     1
3  PGP004     2
4  PGP005     4
5  PGP007     3
6  PGP008     2
7  PGP009     4
8  PGP010     3
  target     pgp well
0  MEIS1  PGP001   A1
1  blank  PGP002   A2
2   TBX2  PGP003   A3
3   EGFR  PGP004   A4
4    CD8  PGP005   A5
5    CD8  PGP007   A6
6  IL2RA  PGP008   A7
7    ATM  PGP009   A8
8  TOP2A  PGP010   A9


In [ ]:
source_sites = list(set(source.site))
mlprep_sites =  [x for x in mlprep_sites if x not in source_sites]

wells_cycle=list(itertools.islice(itertools.cycle(wells), num_cellgos))
well_rows = list(itertools.chain.from_iterable([itertools.repeat(item, count) for item, count in zip(wells_cycle,pgp_combos_fixed_lengths)]))


In [ ]:
if num_cellgos > 192
 raise ValueError('You can construct a max 192 cellgorithms ') 